In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model

# Load MovieLens 100K dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
data = pd.read_csv(url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# Parameters
latent_dim = 20
num_users = data['user_id'].nunique()  # Number of unique users
num_items = data['item_id'].nunique()  # Number of unique items
batch_size = 64
epochs = 50

# Normalize the ratings to be between 0 and 1
ratings = data['rating'].values
ratings = (ratings - ratings.min()) / (ratings.max() - ratings.min())

# Create trainable embeddings for users and items
user_embedding_layer = layers.Embedding(input_dim=num_users, output_dim=latent_dim, input_length=1)
item_embedding_layer = layers.Embedding(input_dim=num_items, output_dim=latent_dim, input_length=1)

# Create the interaction matrix (user-item interactions)
interaction_matrix = np.zeros((num_users, num_items))
for i, row in data.iterrows():
    user_idx = row['user_id'] - 1  # User ID starts at 1, so subtract 1 for indexing
    item_idx = row['item_id'] - 1  # Item ID starts at 1, so subtract 1 for indexing
    interaction_matrix[user_idx, item_idx] = ratings[i]

# Generator model
def build_generator(latent_dim, num_items):
    user_id_input = layers.Input(shape=(1,))  # User ID as input
    user_embedding = user_embedding_layer(user_id_input)  # Lookup embedding
    user_embedding = layers.Flatten()(user_embedding)
    
    noise_input = layers.Input(shape=(latent_dim,))  # Random noise
    merged = layers.Concatenate()([user_embedding, noise_input])
    
    x = layers.Dense(128, activation='relu')(merged)
    x = layers.Dense(256, activation='relu')(x)
    generated_items = layers.Dense(num_items, activation='sigmoid')(x)
    
    return Model([user_id_input, noise_input], generated_items)

# Discriminator model
def build_discriminator(latent_dim, num_items):
    user_id_input = layers.Input(shape=(1,))  # User ID as input
    user_embedding = user_embedding_layer(user_id_input)  # Lookup embedding
    user_embedding = layers.Flatten()(user_embedding)
    
    item_input = layers.Input(shape=(num_items,))  # Item interaction matrix
    merged = layers.Concatenate()([user_embedding, item_input])
    
    x = layers.Dense(256, activation='relu')(merged)
    x = layers.Dense(128, activation='relu')(x)
    validity = layers.Dense(1, activation='sigmoid')(x)
    
    return Model([user_id_input, item_input], validity)

# Build models
generator = build_generator(latent_dim, num_items)
discriminator = build_discriminator(latent_dim, num_items)

# Optimizers
optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

# Compile discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Build and compile the combined model
discriminator.trainable = False
user_input = layers.Input(shape=(1,))
noise_input = layers.Input(shape=(latent_dim,))
generated_items = generator([user_input, noise_input])
validity = discriminator([user_input, generated_items])
combined = Model([user_input, noise_input], validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# Training loop
for epoch in range(epochs):
    # Select a random batch of user IDs
    user_ids = np.random.randint(0, num_users, batch_size)
    
    # Generate random noise
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    # Generate fake item interactions
    fake_items = generator.predict([user_ids, noise])
    
    # Get real item interactions from interaction matrix
    real_items = interaction_matrix[user_ids]
    
    # Labels for real and fake data
    real = np.ones((batch_size, 1)) * 0.9  # Smoothed labels
    fake = np.zeros((batch_size, 1)) + 0.1
    
    # Train discriminator
    d_loss_real = discriminator.train_on_batch([user_ids, real_items], real)
    d_loss_fake = discriminator.train_on_batch([user_ids, fake_items], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train generator
    g_loss = combined.train_on_batch([user_ids, noise], real)
    
    # Print progress
    print(f"Epoch {epoch + 1}/{epochs} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")


C:\Users\renis\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step 


C:\Users\renis\AppData\Roaming\Python\Python312\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 1/50 [D loss: 0.7834852933883667 | D accuracy: 0.0] [G loss: [array(0.97387266, dtype=float32), array(0.97387266, dtype=float32), array(0., dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 2/50 [D loss: 0.9154446125030518 | D accuracy: 0.0] [G loss: [array(0.98364806, dtype=float32), array(0.98364806, dtype=float32), array(0., dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 3/50 [D loss: 0.9486445188522339 | D accuracy: 0.0] [G loss: [array(0.99291867, dtype=float32), array(0.99291867, dtype=float32), array(0., dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 20/50 [D loss: 1.1480705738067627 | D accuracy: 0.0] [G loss: [array(1.1588192, dtype=float32), array(1.1588192, dtype=float32), array(0., dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Epoch 21/50 [D loss: 1.1552355289459229 | D accuracy: 0.0] [G loss: [array(1.1656545, dtype=float32), array(1.1656545, dtype=float32), array(0., dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste